In [1]:
import awkward as ak
import hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection
import json
import numpy as np
import mplhep as hep
import matplotlib.pyplot as plt

In [2]:
filename = "/home/prayag/Coffea/data/3BF5BC57-5D24-1C4D-B277-7B0BAA5DF3C7.root"
events = NanoEventsFactory.from_root(
    filename,
    schemaclass=NanoAODSchema.v7
).events()

In [5]:
from monoHbbtools import Load

In [4]:
def getDataset(keymap,load=True, dict = None, files=None, begin=0, end=0, mode = "sequential"):
    #Warning : Never use 'files' with 'begin' and 'end'
    if load :
        fileset = Load.Loadfileset("../monoHbbtools/Load/newfileset.json")
        fileset_dict = fileset.getraw()
    else :
        fileset_dict = dict
    MCmaps = ["ZJets_NuNu"]

    
    runnerfileset = Load.buildFileset(fileset_dict[keymap],"fnal")
    flat_list={}
    flat_list[keymap] = []

    if mode == "sequential":
        if end - begin <= 0:
            print("Invalid begin and end values.\nFalling back to full dataset...")
            outputfileset = runnerfileset
        else:
            # for key in runnerfileset.keys() :
            #     flat_list[keymap] += runnerfileset[key]
            #indexer
            index={}
            i = 1
            for key in runnerfileset.keys() :
                index[key] = []
                for file in runnerfileset[key] :
                    index[key].append(i)
                    i += 1

            accept = np.arange(begin,end+1,1)
            print(accept)
            temp = {}
            for key in runnerfileset.keys() :
                temp[key] = []
                for i in range(len(runnerfileset[key])) :
                    if index[key][i] in accept :
                        temp[key].append(runnerfileset[key][i])
            #outputfileset = {keymap : flat_list[keymap][(begin - 1) :end]}
            #outputfileset = {keymap : temp}
            outputfileset = temp
    elif mode == "divide" :
        if files == None:
            print("Invalid number of files.\nFalling back to full dataset...")
            outputfileset = runnerfileset
        else:
            # Divide the share of files from all the 8 categories of ZJets_NuNu
            file_number = 0
            while file_number < files :
                for key in runnerfileset.keys():
                    if file_number >= files :
                        break
                    flat_list[keymap] += [runnerfileset[key][0]]
                    runnerfileset[key] = runnerfileset[key][1:]
                    file_number += 1
            outputfileset = {keymap : flat_list[keymap]}
    else:
        print("Invalid mode of operation", mode)
        raise KeyError
    
    print("Running ", np.array([len(value) for value in outputfileset.values()]).sum(), " files...")
    return outputfileset

In [6]:
getDataset("MET_Run2018",load=True,mode="sequential",begin=1,end=10)

[ 1  2  3  4  5  6  7  8  9 10]
Running  10  files...


{'MET_Run2018A': ['root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/0F8C0C8C-63E4-1D4E-A8DF-506BDB55BD43.root',
  'root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/10C73E73-0C15-2F4B-9E0B-E3DE1C54A597.root',
  'root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/1E8B7F5A-4B29-8F46-B2E1-549805E5CBB2.root',
  'root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/1ED54027-5256-064F-A776-C10A0920256A.root',
  'root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/4480C919-0BD7-B74D-8B6A-BFF37A1DB4DF.root',
  'root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/4C8B6DD1-441E-1E4C-87E9-B7D97060E508.root',
  'root://cmsxrootd.fnal.gov///store/data/Run2018A/MET/NANOAOD/UL2018_MiniAODv2_NanoAODv9-v2/110000/5098B0BA-B20A-6D4C-B